# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.126564e+02     1.759567e+00
 * time: 0.08927083015441895
     1     1.167548e+01     8.905493e-01
 * time: 0.9449608325958252
     2    -1.188188e+01     1.063145e+00
 * time: 1.129241943359375
     3    -3.401769e+01     7.776548e-01
 * time: 1.27518892288208
     4    -4.774330e+01     5.771220e-01
 * time: 1.435628890991211
     5    -5.695756e+01     2.362981e-01
 * time: 1.586568832397461
     6    -5.982264e+01     1.179468e-01
 * time: 1.6943278312683105
     7    -6.087088e+01     5.657247e-02
 * time: 1.8085639476776123
     8    -6.126893e+01     8.486107e-02
 * time: 1.9168219566345215
     9    -6.161905e+01     3.234443e-02
 * time: 2.031972885131836
    10    -6.183528e+01     2.774569e-02
 * time: 2.1420300006866455
    11    -6.199004e+01     1.875308e-02
 * time: 2.2571489810943604
    12    -6.205339e+01     1.639865e-02
 * time: 2.365466833114624
    13    -6.211099e+01     1.734358e-02
 * time: 2.4780778884887695

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557684
    AtomicNonlocal      14.8522653
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485373 
    Xc                  -19.3336821

    total               -62.261666459473
